#### PIPE - 1
This pipe covers the following :
1. Implementing strong tracker
2. Implementing speed tracking

In [1]:
import cv2
import numpy as np
import pandas as pd
import torch
import time
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *

In [ ]:
down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines

counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6



tracker=tracker_init(reid_model_path='models/',cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('yolov8s.pt')
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'
vid = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
while True:    
    ret, im = vid.read()
    if not ret:
        break
    
    im=cv2.resize(im,(1020, 500))
    
    results=model.predict(im,conf=0.4,verbose=False,device=[1],classes=[2,3])
    
    im=line_plotter(frame=im,x_start=100,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    im=line_plotter(frame=im,x_start=8,x_end=927,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets=tracker_element_creator(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, im) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                im,object_id,cx,cy=plot_tracks(track,im)
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            # print("entered if 5")
                             
                            counter_down.append(object_id) 
    
                            distance = 9
                            # print("entered if 6")
                             
                            a_speed_kh=speed_calculator_kmph(distance,elapsed_time)
                            # print("entered if 7")
                             
                            print(f"for {object_id} speed is -> {a_speed_kh}")
        
    out_main.write(im)
    # # break on pressing q or spaceq
    cv2.imshow('BoxMOT detection', im)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

2024-04-29 15:06:00.074 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"


for 5 speed is -> 17
for 10 speed is -> 20
for 28 speed is -> 18
for 34 speed is -> 20


In [7]:
dets=[]
if dets:
    print("EMty")
else:
    print("workign")

workign
